# svm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [6]:
# drop the unnecessary columns
df.drop(["deck", "embark_town", "alive", "class", "who", "adult_male"], axis=1, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    object 
 3   age       714 non-null    float64
 4   sibsp     891 non-null    int64  
 5   parch     891 non-null    int64  
 6   fare      891 non-null    float64
 7   embarked  889 non-null    object 
 8   alone     891 non-null    bool   
dtypes: bool(1), float64(2), int64(4), object(2)
memory usage: 56.7+ KB


Now fill the values

In [8]:
# fill the missing values in "age" with mean
df["age"].fillna(df["age"].mean(), inplace=True)

In [9]:
# As there are 889 values out of 891 in "embarked". To correct it, we will remove that 2 null rows from dataset
df.dropna(subset=["embarked"], inplace=True)

In [10]:
df.info()
# here, now all the values are equal

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  889 non-null    int64  
 1   pclass    889 non-null    int64  
 2   sex       889 non-null    object 
 3   age       889 non-null    float64
 4   sibsp     889 non-null    int64  
 5   parch     889 non-null    int64  
 6   fare      889 non-null    float64
 7   embarked  889 non-null    object 
 8   alone     889 non-null    bool   
dtypes: bool(1), float64(2), int64(4), object(2)
memory usage: 63.4+ KB


# Encoding

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [12]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,True
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,True


In [13]:
# here, we will encode the "sex" and "embarked"
df["sex"] = le.fit_transform(df['sex'])
df["embarked"] = le.fit_transform(df['embarked'])
# here, we did the label encoding. So the values will convert in 0, 1, 2, ...

In [14]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,1,22.0,1,0,7.2500,2,False
1,1,1,0,38.0,1,0,71.2833,0,False
2,1,3,0,26.0,0,0,7.9250,2,True
3,1,1,0,35.0,1,0,53.1000,2,False
4,0,3,1,35.0,0,0,8.0500,2,True


In [15]:
# convert all the string values in int
df = df.astype(int)

In [16]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,1,22,1,0,7,2,0
1,1,1,0,38,1,0,71,0,0
2,1,3,0,26,0,0,7,2,1
3,1,1,0,35,1,0,53,2,0
4,0,3,1,35,0,0,8,2,1


# model

In [17]:
# separated the inputs(from which we want to predict) and output(that we want to predict)
X = df.drop("survived", axis=1)
y = df["survived"]

**train-test split**

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**feature scaling**

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [20]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

**implement knn**

In [21]:
from sklearn.svm import SVC
model = SVC(kernel='rbf')

# train
model.fit(X_train_scaled, y_train)

SVC()

**make predictions**

In [22]:
y_predict = model.predict(X_test_scaled)

# model evaluation

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [24]:
# aaccuracy_score
accuracy_score(y_test,y_predict)

0.8258426966292135

In [25]:
# coconfusion_matrix
confusion_matrix(y_test, y_predict)

array([[96, 13],
       [18, 51]])

In [26]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       109
           1       0.80      0.74      0.77        69

    accuracy                           0.83       178
   macro avg       0.82      0.81      0.81       178
weighted avg       0.82      0.83      0.82       178



# cross validation

In [32]:
# here is the X, that we created above by splitting y
X

,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,3,1,22,1,0,7,2,0
1,1,0,38,1,0,71,0,0
2,3,0,26,0,0,7,2,1
3,1,0,35,1,0,53,2,0
4,3,1,35,0,0,8,2,1
...,...,...,...,...,...,...,...,...
886,2,1,27,0,0,13,2,1
887,1,0,19,0,0,30,2,1
888,3,0,29,1,2,23,2,0
889,1,1,26,0,0,30,0,1


In [33]:
# here is y, that we split out from df
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: survived, Length: 889, dtype: int64

**feature scaling**

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled =scaler.fit_transform(X)

**cross validation**

In [38]:
# import library for cross validation
from sklearn.model_selection import cross_val_score

In [42]:
scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
# here, we will input the model, on which we want to perform cross validations, like svm, knn, etc

In [43]:
# here is the 5 different accuracy scores, as we put cv=5
print(scores)

[0.83146067 0.82022472 0.81460674 0.80898876 0.86440678]


In [44]:
# to find the average accuracy score
print(scores.mean())

0.8279375357074844
